In [1]:
import pypelines.ts_forecast_pipeline as pipe
from pypelines import utils
import pandas as pd
airline = pd.read_csv("pypelines/datasets/timeseries_forecast/airline.csv")

In [2]:
airline


,Date,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


In [3]:
tsf = pipe.TSForecastPipeline(data = "airline",models=['Exponential Smoothing'],
                 date_column = 'Date',
                 date_format = "%Y-%m",
                 frequency = "M",
                 forecast_horizon = [1,2,3])

In [5]:
tsf.code_to_clipboard()

In [12]:


from sktime import *
from sklearn.metrics import mean_squared_error


import pandas as pd
import numpy as np


date_col = 'Date'
features = list(airline.columns)
feature_df = airline[features]

x_train = airline[features]
x_train_preprocessed = x_train
x_train_preprocessed[date_col] = pd.to_datetime(x_train_preprocessed[date_col],format = "%Y-%m")
x_train_preprocessed = x_train_preprocessed.set_index(date_col)
x_train_preprocessed.index = x_train_preprocessed.index.to_period(freq = "M")

model_comparison_list = []

##### End of Data Processing Pipeline #####


##### Model Pipeline for Exponential Smoothing #####

from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.model_selection import ExpandingWindowSplitter,ForecastingGridSearchCV,ExpandingWindowSplitter
from sklearn.metrics import mean_squared_error,make_scorer,r2_score,explained_variance_score
import matplotlib.pyplot as plt
cv = ExpandingWindowSplitter(fh=[1, 2, 3])

ExponentialSmoothing_param_grid = {
"sp": np.arange(12, 13, 1),
}


ExponentialSmoothing_model = ExponentialSmoothing()

ExponentialSmoothing_gscv = ForecastingGridSearchCV(
    forecaster=ExponentialSmoothing_model,
    param_grid=ExponentialSmoothing_param_grid,
    cv=cv,
    verbose=1)
ExponentialSmoothing_gscv.fit(x_train_preprocessed)

# get the prediction on the test data
ExponentialSmoothing_y_pred = ExponentialSmoothing_gscv.predict(fh=[1, 2, 3])
##### End of Model Pipeline for Exponential Smoothing #####
##### Model Comparison #####



Fitting 132 folds for each of 1 candidates, totalling 132 fits


c:\Users\Kreshnaa Raam\AppData\Local\Programs\Python\Python310\lib\site-packages\sktime\forecasting\model_evaluation\_functions.py:413: FitFailedWarning: 
                In evaluate, fitting of forecaster ExponentialSmoothing failed,
                you can set error_score='raise' in evaluate to see
                the exception message. Fit failed for len(y_train)=10.
                The score will be set to nan.
                Failed forecaster with parameters: ExponentialSmoothing(sp=12).
                
  result = _evaluate_window(
c:\Users\Kreshnaa Raam\AppData\Local\Programs\Python\Python310\lib\site-packages\sktime\forecasting\model_evaluation\_functions.py:413: FitFailedWarning: 
                In evaluate, fitting of forecaster ExponentialSmoothing failed,
                you can set error_score='raise' in evaluate to see
                the exception message. Fit failed for len(y_train)=11.
                The score will be set to nan.
                Failed forecaster wi

NotFittedError: All fits of forecaster failed,
                set error_score='raise' to see the exceptions.
                Failed forecaster: ExponentialSmoothing()

In [8]:
ExponentialSmoothing_param_grid = {
"sp": np.arange(12, 13, 1),
}

In [9]:
ExponentialSmoothing_param_grid

{'sp': array([12])}

In [13]:
from sktime.datasets import load_airline
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
y = load_airline()
forecaster = ExponentialSmoothing(
    trend='add', seasonal='multiplicative', sp=12
)  
forecaster.fit(y)  

y_pred = forecaster.predict(fh=[1,2,3]) 

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "__init__.pxd", line 942, in numpy.import_array
RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem .

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Kreshnaa Raam\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Kreshnaa Raam\AppData\Local\Temp\ipykernel_597156\3746879201.py", line 7, in <module>
    forecaster.fit(y)
  File "c:\Users\Kreshnaa Raam\AppData\Local\Programs\Python\Python310\lib\site-packages\sktime\forecasting\base\_base.py", line 348, in fit
    self._fit(y=y_inner, X=X_inner, fh=f